In [1]:
from keras.applications import inception_v3
from keras import backend as K

K.set_learning_phase(0)
model = inception_v3.InceptionV3(weights ='imagenet', include_top =False)

Using TensorFlow backend.


In [2]:
layer_contributions ={'mixed2':0.2,'mixed3':3.,'mixed4':2.,'mixed5':1.5}
layer_dict = dict([(layer.name,layer) for layer in model.layers])
# difing loss to maximised
loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff =layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    scaling =K.prod(K.cast(K.shape(activation),'float32'))
    loss += coeff* K.sum(K.square(activation[:,2:-2,2:-2,:]))/scaling

In [3]:
# gradiant -accent process
dream = model.input
grads = K.gradients(loss,dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)),1e-7)

outputs = [loss, grads]
fetch_loss_and_grads =K.function([dream],outputs)

def eval_loass_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
def gradient_ascent(x, iterations, step, max_loss =None):
    for i in range (iterations):
        loss_value, grad_values = eval_loass_grads(x)
        if max_loss is not None and loss_value> max_loss:
            break
        print('....Loss value at',i,':', loss_value)
        x+= step*grad_values
    return x
    

In [6]:
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img =np.copy(img)
    factors = (1,
               float(size[0])/img.shape[1],
               float(size[1])/img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors,order = 1)
def save_img(img, fname):
    pil_img = deprocess_img(np.copy(img))
    scipy.misc.imsave(fname,pil_img)
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    
    img = inception_v3.preprocess_input(img)
    return img
def deprocess_img(x):
    if K.image_data_format() == 'channels_first':
        x =x.reshape((3, x.shape[2], x.shape[3]))
        x.transpose((1,2,0))
    else:
        x =x.reshape((x.shape[1], x.shape[2],3))
    x /=2.
    x +=0.5
    x *=255.
    x = np.clip(x, 0, 255)
    return x

In [7]:
#running  gradiant ascent over diff. successive scales
import numpy as np 
step =.005
num_octave =5
octave_scale=1.5
iteration = 20

max_loss=10.

base_img_path = './Data_Sets/w.jpeg'

img = preprocess_image(base_img_path)
origanl_shape =img.shape[1:3]
successive_shapes = [origanl_shape]
for i in range (1,num_octave):
    shape = tuple([int(dim/(octave_scale **i)) for dim in origanl_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_origanl_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('Processing image shape', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img, iterations = iteration, 
                         step =step,
                         max_loss=max_loss)
    upscaled_shrunk_org_img = resize_img(shrunk_origanl_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_org_img
    
    img += lost_detail
    shrunk_origanl_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_'+str(shape)+'.png')
save_img(img, fname='dream.png')

Processing image shape (205, 115)
....Loss value at 0 : 0.569526
....Loss value at 1 : 0.69642
....Loss value at 2 : 0.86199
....Loss value at 3 : 1.02411
....Loss value at 4 : 1.1942
....Loss value at 5 : 1.37419
....Loss value at 6 : 1.55397
....Loss value at 7 : 1.72567
....Loss value at 8 : 1.90747
....Loss value at 9 : 2.02874
....Loss value at 10 : 2.22579
....Loss value at 11 : 2.34697
....Loss value at 12 : 2.52133
....Loss value at 13 : 2.70827
....Loss value at 14 : 2.86422
....Loss value at 15 : 3.02256
....Loss value at 16 : 3.17851
....Loss value at 17 : 3.29972
....Loss value at 18 : 3.46899
....Loss value at 19 : 3.60415


C:\Users\DELL\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  del sys.path[0]


Processing image shape (308, 173)
....Loss value at 0 : 1.76431
....Loss value at 1 : 2.44819
....Loss value at 2 : 3.01717
....Loss value at 3 : 3.51694
....Loss value at 4 : 3.98719
....Loss value at 5 : 4.44178
....Loss value at 6 : 4.83912
....Loss value at 7 : 5.24948
....Loss value at 8 : 5.64633
....Loss value at 9 : 5.99701
....Loss value at 10 : 6.3534
....Loss value at 11 : 6.64818
....Loss value at 12 : 6.9743
....Loss value at 13 : 7.2602
....Loss value at 14 : 7.55295
....Loss value at 15 : 7.8204
....Loss value at 16 : 8.07412
....Loss value at 17 : 8.34555
....Loss value at 18 : 8.53418
....Loss value at 19 : 8.76168
Processing image shape (462, 260)
....Loss value at 0 : 2.38692
....Loss value at 1 : 3.15863
....Loss value at 2 : 3.84042
....Loss value at 3 : 4.46472
....Loss value at 4 : 5.0496
....Loss value at 5 : 5.57298
....Loss value at 6 : 6.05985
....Loss value at 7 : 6.49391
....Loss value at 8 : 6.92856
....Loss value at 9 : 7.31622
....Loss value at 10 : 7.72